In [1]:
import cpp_soft_info

# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()

In [4]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 30
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
258,2023-10-30 09:45:17.340386+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpt78vcq70008qf1ag,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...
76,2023-10-29 14:47:58.814875+01:00,bigger_rep_codes,ibm_sherbrooke,cmz653m3r3vg008wf9j0,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,1,_is_hex=True,Run bigger Repetition codes


In [5]:
memories = {}
jobs = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()
    jobs[mmr_name] = job_id

memory_str = "mmr_log_0"
memory = memories[memory_str][:]
job_id = jobs[memory_str]
print(memory.shape)

# Initialize the code

In [ ]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid

d = max_distance
synd_rounds = max_distance

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=synd_rounds,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend(DEVICE), _is_hex=True)
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes

In [ ]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-22T10:30:00" # "2023-11-22T" works too
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

In [ ]:
# get_counts
counts = cpp_soft_info.get_counts(memory, qubit_mapping, grid_dict, processed_scaler_dict, synd_rounds)

# Decode

In [ ]:
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict, p_data=6e-3, p_mixed=6e-3, #p_mixed=1e-80, for d=30
                                           common_measure=-1)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 5s

num_errors: 13 out of 1111 shots


In [ ]:
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

num_errors: 46 out of 1111 shots


In [ ]:
num_errors = cpp_soft_info.decode_IQ_shots_flat_diag(matching._matching_graph, memory, 
                                           synd_rounds, qubit_mapping, grid_dict, 
                                           processed_scaler_dict, p_mixed=1e-80)
print("num_errors:", num_errors, "out of", len(memory), "shots")

# takes 1s

AttributeError: module 'cpp_soft_info' has no attribute 'decode_IQ_shots_flat_diags'